In [1]:
library("gplots")
library(RColorBrewer)

Warning message:
"package 'gplots' was built under R version 4.0.3"

Attaching package: 'gplots'


The following object is masked from 'package:stats':

    lowess




In [2]:
names<-c("L.donovani","L.major","L.braziliensis")
file.names<-paste(names,"SignificantCholesterolGenes.txt",sep="_")
combined.results<-lapply(file.names,function(x){
        read.table(file=paste0("assets/",x),header=T,sep="\t",quote="")
})
names(combined.results)<-names

In [3]:
head(combined.results[[1]],2)

,ENTREZID,GENENAME,SYMBOL,L.donovani_4vsControl_logFC,L.donovani_4vsControl_test,L.donovani_8vsControl_logFC,L.donovani_8vsControl_test,L.donovani_24vsControl_logFC,L.donovani_24vsControl_test
,<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,10062,nuclear receptor subfamily 1 group H member 3,NR1H3,0.8831344,1,1.4147506,1,1.8102835,1
2,10613,ER lipid raft associated 1,ERLIN1,0.2552710,0,0.5287214,0,0.8387841,1


In [4]:
store<-merge(combined.results[[1]],combined.results[[2]][,-c(2:3)],
              by="ENTREZID")
logFC.heatmap<-store[,c(c(1:3),grep("logFC",colnames(store)))]
head(logFC.heatmap)

,ENTREZID,GENENAME,SYMBOL,L.donovani_4vsControl_logFC,L.donovani_8vsControl_logFC,L.donovani_24vsControl_logFC,L.major_4vsControl_logFC,L.major_8vsControl_logFC,L.major_24vsControl_logFC
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1593,cytochrome P450 family 27 subfamily A member 1,CYP27A1,0.6617582,1.1820994,1.8167363,0.3974868,0.9348959,1.8417425
2,2055,CLN8 transmembrane ER and ERGIC protein,CLN8,1.0428514,0.9651887,0.8254485,1.0277801,1.2407114,0.8131458
3,2222,farnesyl-diphosphate farnesyltransferase 1,FDFT1,-0.6235646,-0.8579507,-0.2867595,-0.3333820,-0.8968429,-0.6231387
4,2230,ferredoxin 1,FDX1,0.4468257,0.8012325,1.2851679,0.5277191,0.8919982,1.4925701
5,4047,lanosterol synthase,LSS,1.0325449,1.3555131,1.2190693,1.3523760,1.7870638,1.4371153
6,4790,nuclear factor kappa B subunit 1,NFKB1,0.7861651,0.6636676,0.6113833,1.2044069,1.1220219,0.8210967


In [76]:
colors<-c(c(brewer.pal(8,"Set1")),c(brewer.pal(5,"Set2")))
treatment_time_color<-rep(c(colors[1],colors[2],colors[3]),2)
strain_color<-c(rep(colors[4],3),rep(colors[5],3))
myCols <- cbind(strain_color,treatment_time_color)

store<-logFC.heatmap
input<-data.matrix(store[,-c(1:3)])

input.mean<-apply(input,1,mean)
input.sd<-apply(input,1,sd)
input <- sweep(input, 1, input.mean, "-")
input <- sweep(input, 1, input.sd, "/")

colnames(myCols)[1] <- "Strains"
colnames(myCols)[2] <- "Treatment Time"
strain_names<-c(rep("L.donovani",3),rep("L.major",3))

treatment_times<-c(4,8,24)
treatment_color_options<-c(colors[1],colors[2],colors[3])
strain_color_options<-c(colors[4],colors[5])

# Top genes heatmap
# dendrogram = c("both","row","column","none")
svg(file="SignificantCholesterolGenes_L.dono_L.major_Heatmap.svg")  
                        
par(cex.main=0.8,mar=c(0,15,1.5,0))
heatmap.2(input, trace="column", tracecol="cyan", 
          col=bluered(20),
          labRow=store$SYMBOL,
          labCol=strain_names,
          dendrogram = "row",Colv=NA,
          scale="column",
          cexRow=1,cexCol=1, 
          margins = c(5.4,12), main="Heatmap of Cholesterol Genes",  
          ColSideColors=treatment_time_color
         )
# legend("topright",inset=c(0.01,0.0),legend=c("L.donovani","L.major"),
#        fill=strain_color_options,cex=0.45,title="Strains",
#        box.lwd=0.5,text.width=0.15)
legend("topright",inset=c(0.4,0.09),legend=paste(treatment_times,"hours"),
       fill=treatment_color_options,cex=0.45,title="Infection Time",
       box.lwd=0.5,text.width=0.15)
          
# for exporting svg etc
dev.off()

png 
  2

In [16]:
head(combined.results[[3]],2)

,ENTREZID,PROBEID,GENENAME,SYMBOL,ENSEMBL,logFC,AveExpr,t,P.Value,adj.P.Val,B,test
,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,10484,3561952,"Sec23 homolog A, coat complex II component",SEC23A,ENSG00000100934,-0.6244444,6.770000,-4.279711,0.0004285496,0.01217136,2.5385809,-1
2,10558,3214582,serine palmitoyltransferase long chain base subunit 1,SPTLC1,ENSG00000090054,-0.3188889,7.719444,-3.218785,0.0046459683,0.04083054,0.4125763,-1


In [20]:
store.all<-merge(logFC.heatmap,combined.results[[3]][,c("ENTREZID","logFC")],
              by="ENTREZID")
heatmap<-store.all[,c(c(1:3),grep("logFC",colnames(store.all)))]

In [70]:
colors<-c(c(brewer.pal(8,"Set1")),c(brewer.pal(5,"Set2")))
treatment_time_color<-c(rep(c(colors[1],colors[2],colors[3]),2),
                        colors[4])
strain_color<-c(rep(colors[5],3),rep(colors[6],3),colors[7])
myCols <- cbind(strain_color,treatment_time_color)

store<-store.all
input<-data.matrix(store[,-c(1:3)])

input.mean<-apply(input,1,mean)
input.sd<-apply(input,1,sd)
input <- sweep(input, 1, input.mean, "-")
input <- sweep(input, 1, input.sd, "/")

colnames(myCols)[1] <- "Strains"
colnames(myCols)[2] <- "Treatment Time"
strain_names<-c(rep("L.donovani",3),rep("L.major",3),"L.braziliensis")

treatment_times<-c(rep(c(4,8,24),2),c(72))
treatment_color_options<-c(colors[1],colors[2],colors[3],colors[4])
strain_color_options<-c(colors[5],colors[6],colors[7])

# Top genes heatmap
# dendrogram = c("both","row","column","none")
svg(file="SignificantCholesterolGenes_L.dono_L.major_L.braz_Heatmap.svg")  
par(cex.main=0.8,mar=c(0,15,1.5,1))
heatmap.2(input, trace="column", tracecol="cyan", 
          col=bluered(20),
          labRow=store$SYMBOL,
          labCol=strain_names,
          dendrogram = "row",Colv=NA,
          scale="column",
          cexRow=1,cexCol=1, 
          margins = c(6.1,12), main="Heatmap of Cholesterol Genes",  
          ColSideColors=treatment_time_color
         )
# legend("topright",inset=c(0.4,0.04),
#        legend=c("L.donovani","L.major","L.braziliensis"),
#        fill=strain_color_options,cex=0.45,title="Strains",
#        box.lwd=0.5,text.width=0.15)
legend("topright",inset=c(0.4,0.12),
       legend=paste(c(4,8,24,72),"hours"),
       fill=treatment_color_options,cex=0.45,title="Infection Time",
       box.lwd=0.5,text.width=0.15)
          
# for exporting svg etc
dev.off()

png 
  2

## Venn Diagram

In [39]:
library("VennDiagram")

Warning message:
"package 'VennDiagram' was built under R version 4.0.3"
Loading required package: grid

Loading required package: futile.logger



In [44]:
names<-c("L.donovani","L.major","L.braziliensis")
file.names<-paste(names,"SignificantGenes.txt",sep="_")
significant.results<-lapply(file.names,function(x){
        read.table(file=paste0("assets/",x),header=T,sep="\t",quote="")
})
names(significant.results)<-names

In [45]:
head(significant.results[[1]],3)

,ENTREZID,GENENAME,SYMBOL,L.donovani_4vsControl_logFC,L.donovani_4vsControl_test,L.donovani_8vsControl_logFC,L.donovani_8vsControl_test,L.donovani_24vsControl_logFC,L.donovani_24vsControl_test
,<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,4489,metallothionein 1A,MT1A,3.564736,1,3.937992,1,3.333190,1
2,4502,metallothionein 2A,MT2A,3.545896,1,3.678668,1,3.405996,1
3,4496,metallothionein 1H,MT1H,3.501433,1,3.669528,1,3.203044,1


In [46]:
library(org.Hs.eg.db)

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:parallel':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welco

In [47]:
human_cholesterol_genes<-read.table(file=paste0("assets/","human_cholesterol.txt")
                              ,header=T,sep="\t",quote="")

In [48]:
store<-select(org.Hs.eg.db, 
                keys=human_cholesterol_genes$SYMBOL, 
                keytype="SYMBOL", 
                columns=c("ENTREZID","GENENAME","ENSEMBL"))

human_cholesterol<- merge(human_cholesterol_genes,store,by="SYMBOL")

'select()' returned 1:many mapping between keys and columns



In [49]:
human_cholesterol_id<-as.data.frame(human_cholesterol[!duplicated(human_cholesterol$ENTREZID),]$ENTREZID)
colnames(human_cholesterol_id)<-c("ENTREZID")
head(human_cholesterol_id)

,ENTREZID
,<chr>
1,39
2,47
3,55902
4,55911
5,8542
6,23780


In [50]:
set.seed(12)
set1 ＜- significant.results[[1]][,"ENTREZID"]
set2 ＜- significant.results[[2]][,"ENTREZID"]
set3 ＜- significant.results[[3]][,"ENTREZID"]
set4  <- human_cholesterol_id$ENTREZID

x<-list(set1, set2, set3, set4)
colors ＜- c("#6b7fff", "#c3db0f", "#ff4059", "#2cff21")#, "#de4dff")
cat.names<-c("L.donovani","L.major","L.braziliensis","Cholesterol")
filename<-c('vennDiagram.tiff')
# Make Venn diagram from list of groups
venn.diagram(x = x ,
            category.names = cat.names,
            filename = filename,output=TRUE,
            height = 3000, width = 3000, resolution = 500, 
            imagetype = "tiff" ,
            scaled = FALSE,
            col = "black",
            fill = colors,
            cat.col = colors,
            cat.cex = 1,
            margin = 0.15,
            main = "Significant Cholesterol Genes",
#             sub = "Featuring: rotation and external lines",
            main.cex = 2,
            sub.cex = 1
)

[1] 1

In [52]:
head(significant.results[[1]],3)

,ENTREZID,GENENAME,SYMBOL,L.donovani_4vsControl_logFC,L.donovani_4vsControl_test,L.donovani_8vsControl_logFC,L.donovani_8vsControl_test,L.donovani_24vsControl_logFC,L.donovani_24vsControl_test
,<int>,<chr>,<chr>,<dbl>,<int>,<dbl>,<int>,<dbl>,<int>
1,4489,metallothionein 1A,MT1A,3.564736,1,3.937992,1,3.333190,1
2,4502,metallothionein 2A,MT2A,3.545896,1,3.678668,1,3.405996,1
3,4496,metallothionein 1H,MT1H,3.501433,1,3.669528,1,3.203044,1


In [54]:
head(significant.results[[3]],2)

,PROBEID,ENTREZID,GENENAME,SYMBOL,ENSEMBL,logFC,AveExpr,t,P.Value,adj.P.Val,B,test
,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,3358906,387266,keratin associated protein 5-3,KRTAP5-3,ENSG00000196224,1.591111,6.841111,9.247997,2.351168e-08,4.312629e-05,10.79181,1
2,3358906,57830,keratin associated protein 5-8,KRTAP5-8,ENSG00000241233,1.591111,6.841111,9.247997,2.351168e-08,4.312629e-05,10.79181,1


In [55]:
store.1_2<-merge(significant.results[[1]],significant.results[[2]][,-c(2:3)],by="ENTREZID")
store.2_3<-merge(significant.results[[2]],significant.results[[3]][,c("ENTREZID","logFC")],by="ENTREZID")
store.1_3<-merge(significant.results[[1]],significant.results[[3]][,c("ENTREZID","logFC")],by="ENTREZID")
write.table(store.1_2, 
            file=paste0("Significant_L.dono_L.major.txt"), 
            sep="\t", quote=F, row.names=FALSE)
write.table(store.2_3, 
            file=paste0("Significant_L.major_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)
write.table(store.1_3, 
            file=paste0("Significant_L.dono_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)

In [78]:
chol.store.1_2<-merge(store.1_2,human_cholesterol_id,by="ENTREZID")
chol.store.2_3<-merge(store.2_3,human_cholesterol_id,by="ENTREZID")
chol.store.1_3<-merge(store.1_3,human_cholesterol_id,by="ENTREZID")
write.table(chol.store.1_2, 
            file=paste0("SignificantCholesterol_L.dono_L.major.txt"), 
            sep="\t", quote=F, row.names=FALSE)
write.table(chol.store.2_3, 
            file=paste0("SignificantCholesterol_L.major_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)
write.table(chol.store.1_3, 
            file=paste0("SignificantCholesterol_L.dono_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)

In [79]:
store.1_2_3<-merge(store.1_2,significant.results[[3]][,c("ENTREZID","logFC")],by="ENTREZID")
chol.store.1_2_3<-merge(store.1_2_3,human_cholesterol_id,by="ENTREZID")
write.table(store.1_2_3, 
            file=paste0("Significant_L.dono_L.major_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)
write.table(chol.store.1_2_3, 
            file=paste0("SignificantCholesterol_L.dono_L.major_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)

## All Cholesterol genes that were significant in atleast one infection time

In [11]:
# all significant cholesterol entrezid
store<- sort(c(combined.results[[1]]$ENTREZID,
                       combined.results[[2]]$ENTREZID,
                       combined.results[[3]]$ENTREZID))
unique.store<-store[duplicated(store)]

In [13]:
length(unique.store)

[1] 41

In [14]:
names<-c("L.donovani","L.major","L.braziliensis")
file.names<-paste(names,"CholesterolGenes.txt",sep="_")
cholesterol.results<-lapply(file.names,function(x){
        read.table(file=paste0("assets/",x),header=T,sep="\t",quote="")
})
names(cholesterol.results)<-names

In [18]:
colnames(cholesterol.results[[3]])

[1] "ENTREZID"  "PROBEID"   "GENENAME"  "SYMBOL"    "ENSEMBL"   "logFC"    
 [7] "AveExpr"   "t"         "P.Value"   "adj.P.Val" "B"         "test"

In [19]:
col.match<- function(colnames,match){
    col.names<-c()
    for(col in match)
      col.names<-c(col.names,grep(col,colnames))
    return(col.names)
}

In [43]:
combined.cholesterol<-data.frame()
names<-names(cholesterol.results)    
for(x in names){
    if(which(x==names)==1){
        col.names<- col.match(colnames(cholesterol.results[[x]]),
                             c("ENTREZID","GENENAME","SYMBOL","logFC","test"))
        combined.cholesterol<-cholesterol.results[[x]][,col.names]
        } 
    else{
        col.names<- col.match(colnames(cholesterol.results[[x]]),
                             c("ENTREZID","logFC","test"))
        combined.cholesterol<-merge(combined.cholesterol,
                     cholesterol.results[[x]][,col.names],
                     by="ENTREZID")
    }
}
braz.col<-colnames(combined.cholesterol)[c(16,17)]
colnames(combined.cholesterol)[c(16,17)]<-paste("L.braziliensis_72vsControl",
                                      braz.col, sep="_")

In [45]:
head(combined.cholesterol)
dim(combined.cholesterol)

,ENTREZID,GENENAME,SYMBOL,L.donovani_4vsControl_logFC,L.donovani_8vsControl_logFC,L.donovani_24vsControl_logFC,L.donovani_4vsControl_test,L.donovani_8vsControl_test,L.donovani_24vsControl_test,L.major_4vsControl_logFC,L.major_8vsControl_logFC,L.major_24vsControl_logFC,L.major_4vsControl_test,L.major_8vsControl_test,L.major_24vsControl_test,L.braziliensis_72vsControl_logFC,L.braziliensis_72vsControl_test
,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<int>
1,39,acetyl-CoA acetyltransferase 2,ACAT2,0.7617580,0.85204904,1.2121357,0,0,0,1.2651908,1.1470215,0.9422972,1,0,0,-0.63777778,-1
2,47,ATP citrate lyase,ACLY,-0.3647168,-0.36479191,-0.1286198,0,0,0,-0.2369586,-0.3042742,-0.3000985,0,-1,0,-0.36666667,-1
3,1387,CREB binding protein,CREBBP,0.0559148,0.21281145,-0.1493207,0,0,0,0.3267746,0.4665610,0.2694436,0,0,0,-0.21000000,-1
4,1593,cytochrome P450 family 27 subfamily A member 1,CYP27A1,0.6617582,1.18209936,1.8167363,0,0,1,0.3974868,0.9348959,1.8417425,0,0,1,0.01444444,0
5,1595,cytochrome P450 family 51 subfamily A member 1,CYP51A1,0.5268071,0.31132528,0.3792897,0,0,0,0.7930525,0.3882755,-0.7371781,0,0,0,-0.45111111,-1
6,1718,24-dehydrocholesterol reductase,DHCR24,0.2293865,-0.06713261,0.7208658,0,0,0,0.1316640,-0.4151133,-1.2324389,0,0,0,-0.44555556,-1


[1] 52 17

In [58]:
colors<-c(c(brewer.pal(8,"Set1")),c(brewer.pal(5,"Set2")))
treatment_time_color<-c(rep(c(colors[1],colors[2],colors[3]),2),
                        colors[4])
strain_color<-c(rep(colors[5],3),rep(colors[6],3),colors[7])
myCols <- cbind(strain_color,treatment_time_color)

store<-combined.cholesterol[,-grep("test",colnames(combined.cholesterol))]
input<-data.matrix(store[,-c(1:3)])

input.mean<-apply(input,1,mean)
input.sd<-apply(input,1,sd)
input <- sweep(input, 1, input.mean, "-")
input <- sweep(input, 1, input.sd, "/")

colnames(myCols)[1] <- "Strains"
colnames(myCols)[2] <- "Treatment Time"
strain_names<-c(rep("L.donovani",3),rep("L.major",3),"L.braziliensis")

treatment_times<-c(4,8,24,72)
treatment_color_options<-c(colors[1],colors[2],colors[3],colors[4])
strain_color_options<-c(colors[5],colors[6],colors[7])

# Top genes heatmap
# dendrogram = c("both","row","column","none")
svg(file="SignificantCholesterolGenes_L.dono_L.major_L.braz_Heatmap.svg")  
                        
par(cex.main=0.8,mar=c(0,15,1.5,0))
heatmap.2(input, trace="column", tracecol="cyan", 
          col=bluered(20),
          labRow=store$SYMBOL,
          labCol=strain_names,
          dendrogram = "row",Colv=NA,
          scale="column",
          cexRow=0.5,cexCol=0.8, 
          margins = c(5.4,12), main="Heatmap of Cholesterol Genes",  
          ColSideColors=treatment_time_color
         )
# legend("topright",inset=c(0.01,0.0),legend=c("L.donovani","L.major"),
#        fill=strain_color_options,cex=0.45,title="Strains",
#        box.lwd=0.5,text.width=0.15)
legend("topright",inset=c(0.4,0.09),legend=paste(treatment_times,"hours"),
       fill=treatment_color_options,cex=0.45,title="Infection Time",
       box.lwd=0.5,text.width=0.15)
          
# for exporting svg etc
dev.off()

png 
  2

In [59]:
write.table(combined.cholesterol, 
            file=paste0("SignificantCholesterol_L.dono_L.major_L.braz.txt"), 
            sep="\t", quote=F, row.names=FALSE)